In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=90340ba7fc1e8cca8137833865830df32a777bc58f0bf7053252339964237a40
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


## NLP Code Along
For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv("/content/sample_data/SMSSpamCollection",inferSchema=True,sep='\t')

In [4]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



# Clean and Prepare the Data
**Create a new length feature:**

In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length',length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [10]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

In [11]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [13]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model
We'll use Naive Bayes, but feel free to play around with this choice!

In [14]:
from pyspark.ml.classification import NaiveBayes

In [15]:
# Use defaults
nb = NaiveBayes()

## Pipeline

In [16]:
from pyspark.ml import Pipeline

In [17]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [18]:
cleaner = data_prep_pipe.fit(data)

In [19]:
clean_data = cleaner.transform(data)

## Training and Evaluation!

In [20]:
clean_data = clean_data.select(['label','features'])

In [21]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [22]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [23]:
spam_predictor = nb.fit(training)

In [24]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [25]:
test_results = spam_predictor.transform(testing)

In [26]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-795.89806455472...|[1.0,5.0955892061...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-219.25474433894...|[1.0,7.9641908156...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-687.39839001107...|[1.0,6.5233560010...|       0.0|
|  0.0|(13424,[0,1,881,1...|[-96.094542427258...|[0.99999996823943...|       0.0|
|  0.0|(13424,[0,1,881,1...|[-97.770795674263...|[0.99999997624772...|       0.0|
|  0.0|(13424,[0,2,3,4,6...|[-1275.1850912720...|[1.0,1.1459225122...|       0.0|
|  0.0|(13424,[0,2,3,6,9...|[-3300.0144446522...|[1.0,5.8232325047...|       0.0|
|  0.0|(13424,[0,2,3,8,2...|[-1595.0492173989...|[1.0,2.2236532630...|       0.0|
|  0.0|(13424,[0,2,4,5,1...|[-2493.7295158704...|[1.0,1.5226616756...|       0.0|
|  0.0|(13424,[0

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9213474174330214


Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!